In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [ ]:
c0 = 1
c1 = 0
target_state = np.array([[c0],[c1]])
print(target_state)

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
def bloch_sphere(circuit):
    state = Statevector(circuit)
    plot_bloch_multivector(state)
def circuit(params):
    device = qml.device('qiskit.aer', wires=1, backend='qasm_simulator')
    @qml.qnode(device, diff_method='parameter-shift')
    params = np.random.normal(0, np.pi/2, (2), requires_grad=True)

    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    qml
bloch_sphere()

In [ ]:
from qiskit_textbook.widgets import plot_bloch_vector_spherical
theta = 2*np.arccos(c0)
target_expectedValue = np.cos(theta)
phi = 0
coords = [theta, phi, 1]
# print(target_expectedValue)
plot_bloch_vector_spherical(coords)

In [ ]:
def vqa_prep(target_state, device):
    device = qml.device('qiskit.aer', wires=1, backend='qasm_simulator')
    params = np.random.normal(0, np.pi/2, (2), requires_grad=True)
    @qml.qnode(device, diff_method='parameter-shift')
    def circuit(params):
        qml.RX(params[0], wires=0)
        qml.RY(params[1], wires=0)

        return qml.expval(qml.PauliZ(0))

In [ ]:
device = qml.device('qiskit.aer', wires=1, backend='qasm_simulator')

params = np.random.normal(0, np.pi/2, (2), requires_grad=True)

@qml.qnode(device, diff_method='parameter-shift')

def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)

    return qml.expval(qml.PauliZ(0))

fig, ax = qml.draw_mpl(circuit, decimals=2)(params)
plt.show()

In [ ]:
def loss(params):
    L = abs(target_expectedValue - circuit(params))
    return L
opt = qml.AdamOptimizer(stepsize=0.01)

for i in range(256):
    print(loss(params), circuit(params), target_expectedValue)
    params = opt.step(loss, params)

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(1)
qc.rx(params[0].item(),0)
qc.ry(params[1].item(),0)

from qiskit.visualization import visualize_transition
visualize_transition(qc)